In [1]:
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers, losses, metrics
from tensorflow.keras import preprocessing

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import os
import re

from konlpy.tag import Okt

import matplotlib.pyplot as plt

In [7]:
base_dir = 'C:/Users/user/Desktop'  # 경로

excel_file = '8월 18일.xlsx'      # 파일명 

In [8]:
excel_dir = os.path.join(base_dir, excel_file)

excel_dir

'C:/Users/user/Desktop\\8월 18일.xlsx'

In [9]:
df=pd.read_excel(excel_dir, header=0)

x=df['title']
x

0                 테슬라, 불공정 자동차 매매약관 고쳤다
1       테슬라, 전기차 배송 뒤 나몰라라…‘불공정 약관’ 없앤다
2          머스크, 테슬라 주가 기록 경신에 세계 4위 부자로
3      테슬라 주가 1800달러 넘었다, 나스닥은 또 사상 최고치
4            테슬라 주가 2000달러 눈앞…월가 목표가 상향
                     ...               
428                                 NaN
429                                 NaN
430                                 NaN
431                                 NaN
432                                 NaN
Name: title, Length: 433, dtype: object

In [10]:
# 태그 단어
PAD = "<PADDING>"   # 패딩
OOV = "<OOV>"       # 없는 단어(Out of Vocabulary)

# 태그 인덱스
PAD_INDEX = 0
OOV_INDEX = 1

# 데이터 타입
#ENCODER_INPUT  = 0
#DECODER_INPUT  = 1
#DECODER_TARGET = 2

# 한 문장에서 단어 시퀀스의 최대 개수 
max_sequences = 100
max_words=100
# 임베딩 벡터 차원
embedding_dim = 100

# LSTM 히든레이어 차원
lstm_hidden_dim = 128

# 정규 표현식 필터
RE_FILTER = re.compile("=[.,!?\"':;~()]")


In [11]:
# 형태소분석 함수
def pos_tag(sentences):
    
    # KoNLPy 형태소분석기 설정
    tagger = Okt()
    
    # 문장 품사 변수 초기화
    sentences_pos = []
    
    # 모든 문장 반복
    for sentence in sentences:
        # 특수기호 제거
        sentence = re.sub(RE_FILTER, "", str(sentence))
        
        # 배열인 형태소분석의 출력을 띄어쓰기로 구분하여 붙임
        sentence = " ".join(tagger.morphs(sentence))
        sentences_pos.append(sentence)
        
    return sentences_pos

# 기사 제목 마이닝

In [12]:
x_new = pos_tag(x)

sentences = []
sentences.extend(x_new)


words = []

# 단어들의 배열 생성
for sentence in sentences:
    for word in sentence.split():
        words.append(word)

# 길이가 0인 단어는 삭제
words = [word for word in words if len(word) > 0]

# 중복된 단어 삭제
words = list(set(words))

# 제일 앞에 태그 단어 삽입
words[:0] = [PAD, OOV]

#print(words) # words : test,trainset의 모든 단어들이 들어간 list

# 단어와 인덱스의 단어사전 생성
word_to_index = {word: index for index, word in enumerate(words)}  # 단어 : 인덱스값 (인코딩시 사용)  # 단어 : 인덱스값
index_to_word = {index: word for index, word in enumerate(words)}  # 인덱스값 : 단어 (디코딩시 사용)  # 인덱스값 : 단어

x_new


['테슬라 , 불공정 자동차 매매 약관 고쳤다',
 '테슬라 , 전기차 배송 뒤 나 몰라라 … ‘ 불공정 약관 ’ 없앤다',
 '머스크 , 테슬라 주가 기록 경신 에 세계 4 위 부자 로',
 '테슬라 주가 1800달러 넘었다 , 나스닥 은 또 사상 최고 치',
 '테슬라 주가 2000달러 눈앞 … 월 가 목표 가 상향',
 '테슬라 도 내년 부터 충돌 시험 받는다 … 보험료 달라질듯',
 "' 갑질 약관 ' 테슬라 무릎 꿇 린 공정위",
 '테슬라 · 벤츠 질주 … 해외 업체 들 각축 장 된 국내 전기 車 시장',
 '1년 새 주가 8 배 ↑ … 테슬라 시 총 , 삼 성 전자 마저 넘었다',
 "' 주식 쪼개기 ' 테슬라 , 1800달러 돌파 … 11% 폭등 ' 최고 치 경신 '",
 "' 주식 쪼개기 ' 테슬라 , 1800달러 돌파 … 11% 폭등 ' 최고 치 경신 '",
 '테슬라 효과 … 니켈 값 9 개월 來 최고',
 "[ 뉴욕증시 ]' 신고 가 ' 나스닥 만 또 날았다 … 테슬라 11% 폭등",
 "르노 삼성 , EV ' 조 에 ' 출시 … 테슬라 잡는다",
 '[ 속보 ] 테슬라 주가 11% 폭등 … 시 총 , P & G 넘어 월마트 위협',
 "' 손해 는 10만원 만 · 인도 후 에는 외면 ' 테슬라 불공정 약관 시정",
 '" 한국 소비자 를 뭘 로 보고 "… 테슬라 , 불공정 약관 손질',
 "공정위 , 배상 책임 피 한 전기 車 ' 테슬라 ' 불공정 약관 ' 제동 '",
 '머스크 , 테슬라 주가 상승 에 ` 세계 4 위 ` 부자',
 '나스닥 , 사상 최고 치 또 경신 … 테슬라 1,800달러 첫 돌파',
 '\' 유럽 1 위 \' 르노 삼성 조 에 , 韓 상륙 ···" 테슬라 모델 3 넘겠다 "',
 "[ 이데일리 N ] 테슬라 제 친 유럽 판매 1 위 전기차 ' 르노 조 에 ' 韓 상륙 外",
 '위워 크에 데인 손정 의 , ‘ 믿을 맨 ’ 아마존 · 테슬라 에 투자',
 '다우 내리고 , 나스닥 은 최고 치 … 테슬라 11% 폭등 

In [13]:
import operator

lst=[]
for i in range(len(x_new)):
    w=x_new[i]
    w=w.split(' ')
    for j in range(len(w)):
        lst.append(w[j])

d={}
ex_t='[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…...》'

for i in lst:
    if i in ex_t:
        continue
    if i not in d:
        d[i]=1
    else:
        d[i]+=1
        
d=sorted(d.items(), key = lambda item: item[1])
d2={}

for i in range(len(d)):
    d2[d[i][0]]=d[i][1]
    

for k,v in d2.items():
    print('{} : {}'.format(k,v))


뒤 : 1
몰라라 : 1
없앤다 : 1
2000달러 : 1
눈앞 : 1
상향 : 1
부터 : 1
받는다 : 1
달라질듯 : 1
갑질 : 1
무릎 : 1
꿇 : 1
린 : 1
벤츠 : 1
업체 : 1
각축 : 1
된 : 1
1년 : 1
8 : 1
삼 : 1
마저 : 1
효과 : 1
값 : 1
9 : 1
來 : 1
신고 : 1
날았다 : 1
속보 : 1
넘어 : 1
위협 : 1
제동 : 1
돋보기 : 1
빼 : 1
들자 : 1
재빨리 : 1
수정 : 1
넘어선 : 1
SR : 1
정운영 : 1
단돈 : 1
하루 : 1
9조원 : 1
챙겼다 : 1
갑부 : 1
드디어 : 1
SM : 1
대신 : 1
호갱 : 1
맞았다 : 1
국고 : 1
지급 : 1
상반기 : 1
싹 : 1
쓸이 : 1
떠 : 1
넘기던 : 1
액 : 1
4.6조원 : 1
관법 : 1
위반 : 1
잡 : 1
을 : 1
있을까 : 1
성능 : 1
차이나 : 1
무조건 : 1
...` : 1
배짱 : 1
장사 : 1
해온 : 1
기간 : 1
비켜 : 1
휩쓴 : 1
전가 : 1
조사 : 1
정당성 : 1
여부 : 1
대하 : 1
여 : 1
인베 : 1
스팅 : 1
닷컴 : 1
등판 : 1
가성 : 1
비 : 1
승부 : 1
나선 : 1
검토 : 1
백만 : 1
마일 : 1
엔 : 1
수준 : 1
높여야 : 1
300% : 1
오른 : 1
新 : 1
프티 : 1
50 : 1
제외 : 1
특별손해 : 1
이항 : 1
영 : 1
가이드 : 1
젠 : 1
단신 : 1
비전 : 1
펀드 : 1
굴욕 : 1
공룡 : 1
만회 : 1
톱 : 1
천 : 1
잡겠다 : 1
지적 : 1
자진 : 1
저항 : 1
지금 : 1
뚫어도 : 1
결국 : 1
뚫는다 : 1
산이 : 1
장악 : 1
정의선 : 1
반격 : 1
예고 : 1
특징 : 1
세 : 1
공식 : 1
스타트업 : 1
쓴맛 : 1
대기업 : 1
25 : 1
보유 : 1
성공 : 1
변신 : 1
굳히기 : 1
나선다 : 1
오르고 : 1
영향 : 1
12억 : 

In [14]:
res=pd.DataFrame(list(d2.items()),
                   columns=['단어', '빈도수'])
res.to_excel('넥슨_텍스트마이닝_제목만2.xlsx')